In [ ]:
import rdkit
from rdkit.Chem import AllChem
import pandas as pd
import os
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit import Chem
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import Draw
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import DataStructs

In [ ]:
import json
with open('../../sup_info/crossdock2020_duplicated_uniprotId_map_smiles_in_trainset_inchi_map.json','r') as f:
    crossdock2020_duplicated_uniprotId_map_smiles_in_trainset = json.load(f)


In [ ]:

def get_trainset_scaffold(x,ref_map):
    uniprot_id  = x['UniprotID']
    if uniprot_id not in ref_map:
        return []
    else:
        all_find_inchi = [i[1:-1] for i in x.Finded_Scaffolds_Inchi[1:-1].split(', ')]

        all_ref_inchi = ref_map[uniprot_id+'_scaffold'].keys()

        list_dup_inchi = list(set(all_find_inchi).intersection(set(all_ref_inchi)))

        list_dup_scaffolds = set([ref_map[uniprot_id+'_scaffold'][i] for i in list_dup_inchi])

        return list_dup_scaffolds
def get_trainset_smiles(x,ref_map):
    uniprot_id  = x['UniprotID']
    if uniprot_id not in ref_map:
        return []
    else:
        all_find_inchi = [i[1:-1] for i in x.Finded_Inchi[1:-1].split(', ')]

        all_ref_inchi = ref_map[uniprot_id].keys()
        list_dup_inchi = list(set(all_find_inchi).intersection(set(all_ref_inchi)))
        list_dup_smiles = set([ref_map[uniprot_id][i] for i in list_dup_inchi])
    
        return list_dup_smiles


### remove proteins in crossdock


In [ ]:
import json
with open('../../sup_info/UniprotIDs_duplicated_with_crossdock2020.json','r') as f:
    UniprotId_in_crossdock = json.load(f)


In [ ]:
def getScaffoldAndSmilesSeen(all_result_path,UniprotId_in_crossdock,seen=True):
    all_results = os.listdir(all_result_path)
    all_results_pd = []

    for temp_path in all_results:
        model_name = os.path.splitext(temp_path)[0]
        temp_path = os.path.join(all_result_path, temp_path)
        temp_pd = pd.read_csv(temp_path)
        temp_pd['ModelName'] = [model_name]*len(temp_pd)
        
        temp_pd['FindedScaffoldIsInTrainset'] = temp_pd.apply(lambda x :len(get_trainset_scaffold(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)), axis=1)
        temp_pd['FindedSmilesIsInTrainset'] = temp_pd.apply(lambda x :len(get_trainset_smiles(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)), axis=1)
        all_results_pd.append(temp_pd)
        
    merged_result_pd = pd.concat(all_results_pd,axis = 0)#['ModelName'].value_counts()
    
    merged_result_pd['Dupliceted_UniprotID'] = merged_result_pd['UniprotID'].apply(lambda x: x in UniprotId_in_crossdock)
    if seen:
        merged_result_pd = merged_result_pd[merged_result_pd['Dupliceted_UniprotID']]
        print('Number of targets: ',len(merged_result_pd['UniprotID'].unique()))
    else:
        merged_result_pd = merged_result_pd[~merged_result_pd['Dupliceted_UniprotID']]
        print('Number of targets: ',len(merged_result_pd['UniprotID'].unique()))
    
    

    merged_result_pd['Finded_Scaffolds_Num'] = merged_result_pd['Finded_Scaffolds_Num'] - merged_result_pd['FindedScaffoldIsInTrainset']
    merged_result_pd['Finded_Smiles_Num'] = merged_result_pd['Finded_Smiles_Num'] - merged_result_pd['FindedSmilesIsInTrainset']



    scaffold_molecule_pd = merged_result_pd.groupby('ModelName').agg(list)[['Finded_Smiles_Num','Finded_Scaffolds_Num']].reset_index()#.apply(lambda x: sum(x)/120,axis = 1).plot(kind = 'bar',figsize=(10,6))
    scaffold_molecule_pd['Mean_Finded_Scaffolds_Num'] = scaffold_molecule_pd['Finded_Scaffolds_Num'].apply(np.mean)
    scaffold_molecule_pd['Mean_Finded_Smiles_Num'] = scaffold_molecule_pd['Finded_Smiles_Num'].apply(np.mean)
  
    scaffold_molecule_pd['Number_of_Finded_Scaffolds'] = scaffold_molecule_pd['Finded_Scaffolds_Num'].apply(lambda x: np.sum(np.array(x, dtype=bool)))
    scaffold_molecule_pd['Number_of_Finded_Smiles'] = scaffold_molecule_pd['Finded_Smiles_Num'].apply(lambda x: np.sum(np.array(x, dtype=bool)))
    return scaffold_molecule_pd

In [ ]:
all_result_path1 = '/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/Hit_Info_Results'

all_result_path2 ='/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/Hit_Info_Results'

all_result_path3 = '/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/Hit_Info_Results'
scaffold_molecule_pd1_seen = getScaffoldAndSmilesSeen(all_result_path1,UniprotId_in_crossdock,seen=True)
scaffold_molecule_pd2_seen = getScaffoldAndSmilesSeen(all_result_path2,UniprotId_in_crossdock,seen=True)
scaffold_molecule_pd3_seen = getScaffoldAndSmilesSeen(all_result_path3,UniprotId_in_crossdock,seen=True)


In [ ]:
scaffold_molecule_pd_repeats_seen = pd.concat([scaffold_molecule_pd1_seen,scaffold_molecule_pd2_seen,scaffold_molecule_pd3_seen],axis = 0)
scaffold_molecule_pd_repeats_seen = scaffold_molecule_pd_repeats_seen[['ModelName','Number_of_Finded_Scaffolds', 'Number_of_Finded_Smiles']].groupby('ModelName').agg(['mean','std']).reset_index()
scaffold_molecule_pd_repeats_seen.columns = ['ModelName','Number_of_Finded_Scaffolds_mean','Number_of_Finded_Scaffolds_std','Number_of_Finded_Smiles_mean','Number_of_Finded_Smiles_std']
scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)

In [ ]:
scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False).to_csv('/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/mol_scaffold_recovery_seen_protein(remove smiles and scaffold in trainset).csv',index = False)

In [ ]:

scaffold_molecule_pd_seen = scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))
color_list = sns.color_palette("deep", n_colors=10)
model_names = sorted(scaffold_molecule_pd_seen['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}
model_colors = [color_map[name] for name in scaffold_molecule_pd_seen['ModelName']]
bars1 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6, color=model_colors)
bars2 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///", color=model_colors)


_ = plt.xticks(range(len(scaffold_molecule_pd_seen['ModelName'])), scaffold_molecule_pd_seen['ModelName'], rotation=45, ha='right')


for bar in bars1:
    # 
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height+2, f'{round(float(height),3)}', ha='center', va='bottom')
for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height+2, f'{round(float(height),3)}', ha='center', va='bottom')
    

plt.errorbar(scaffold_molecule_pd_seen['ModelName'], scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'], 
             yerr=scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

plt.errorbar(scaffold_molecule_pd_seen['ModelName'], scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'], 
             yerr=scaffold_molecule_pd_seen['Number_of_Finded_Smiles_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

    
    
plt.ylim(0, 1)  # Add 20% padding to the top

plt.title('Seen Proteins(Remove Smiles and Scaffold in Trainset)')

plt.ylabel('Number of Targets')
plt.legend()

plt.show()

In [ ]:

scaffold_molecule_pd_seen = scaffold_molecule_pd_repeats_seen.sort_values(by = 'Number_of_Finded_Scaffolds_mean',ascending = False)
plt.figure(figsize=(10,6))
color_list = sns.color_palette("deep", n_colors=10)
model_names = sorted(scaffold_molecule_pd_seen['ModelName'].tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}
model_colors = [color_map[name] for name in scaffold_molecule_pd_seen['ModelName']]
bars1 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'],label = 'Number of Finded Scaffolds',alpha = 0.6, color=model_colors )
bars2 = plt.bar(scaffold_molecule_pd_seen['ModelName'],scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'],label = 'Number of Finded Smiles',alpha = 0.9,hatch="///", color=model_colors )


_ = plt.xticks(range(len(scaffold_molecule_pd_seen['ModelName'])), scaffold_molecule_pd_seen['ModelName'], rotation=45, ha='right')



plt.errorbar(scaffold_molecule_pd_seen['ModelName'], scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_mean'], 
             yerr=scaffold_molecule_pd_seen['Number_of_Finded_Scaffolds_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

plt.errorbar(scaffold_molecule_pd_seen['ModelName'], scaffold_molecule_pd_seen['Number_of_Finded_Smiles_mean'], 
             yerr=scaffold_molecule_pd_seen['Number_of_Finded_Smiles_std'], fmt='none', color='blue', capsize=1,alpha = 0.5)

    
    
plt.ylim(0, )  # Add 20% padding to the top
# plt.ylabel('Number of Targets')
plt.title('Seen Proteins(Remove Smiles and Scaffold in Trainset)')
# plt.legend()
plt.ylabel('Number of Targets')
# plt.legend()
plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/mol_scaffold_recovery_seen_protein(remove smiles and scaffold in trainset)_no_text.svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()

### compute the similarity between trainset and generated molecules

In [ ]:
# merged_result_pd[merged_result_pd['Finded_Smiles_Num']!=0].iloc[1].Finded_Smiles[1:-1].split(', ')[0][1:-1]=='N[C@@H](Cc1ccc(O)c(O)c1)C(=O)O'
def getScaffoldAndSmilesSeen(all_result_path,UniprotId_in_crossdock,seen=True):
    all_results = os.listdir(all_result_path)
    all_results_pd = []

    for temp_path in all_results:
        model_name = os.path.splitext(temp_path)[0]
        temp_path = os.path.join(all_result_path, temp_path)
        temp_pd = pd.read_csv(temp_path)
        temp_pd['ModelName'] = [model_name]*len(temp_pd)
        
        temp_pd['FindedScaffoldIsInTrainset'] = temp_pd.apply(lambda x :len(get_trainset_scaffold(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)), axis=1)
        temp_pd['FindedSmilesIsInTrainset'] = temp_pd.apply(lambda x :len(get_trainset_smiles(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)), axis=1)
        all_results_pd.append(temp_pd)
        
    merged_result_pd = pd.concat(all_results_pd,axis = 0)#['ModelName'].value_counts()
    merged_result_pd['Dupliceted_UniprotID'] = merged_result_pd['UniprotID'].apply(lambda x: x in UniprotId_in_crossdock)
    if seen:
        merged_result_pd = merged_result_pd[merged_result_pd['Dupliceted_UniprotID']]
        print('Number of targets: ',len(merged_result_pd['UniprotID'].unique()))
    else:
        merged_result_pd = merged_result_pd[~merged_result_pd['Dupliceted_UniprotID']]
        print('Number of targets: ',len(merged_result_pd['UniprotID'].unique()))
    
    return merged_result_pd
scaffold_molecule_pd1_seen = getScaffoldAndSmilesSeen(all_result_path1,UniprotId_in_crossdock,seen=True)

In [ ]:



def get_max_smi_to_trainset(x,ref_map):
    
    uniprot_id  = x['UniprotID']
    ref_smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(i)) for i in ref_map[uniprot_id].values() if i != '']
    Finded_Smiles = [i[1:-1] for i in x.Finded_Smiles[1:-1].split(', ') if i != '']
        # Vectorized calculation of similarities
    ref_mols = [Chem.MolFromSmiles(smiles) for smiles in ref_smiles if smiles != '']
    valid_mols = [mol for mol in ref_mols if mol is not None]
    ref_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in valid_mols]
## 找到ref_smiles 和训练数据分子的相似性最高的分子
    max_similarities = []
    for ref_Finded_smiles in Finded_Smiles:
        if ref_Finded_smiles == '':
            # max_similarities.append(0.0)
            continue
            
        ref_Finded_mol = Chem.MolFromSmiles(ref_Finded_smiles)
        if ref_Finded_mol is None:
            continue
            
        ref_Finded_fp = AllChem.GetMorganFingerprintAsBitVect(ref_Finded_mol, 2, nBits=2048)
        
        similarities = [DataStructs.TanimotoSimilarity(ref_Finded_fp, fp) for fp in ref_fps]
        
        max_similarities.append(max(similarities))
    
    return max_similarities


In [ ]:
# selected_id = 'P24941'
def get_max_scaffold_smi_to_trainset(x,ref_map):
    
    uniprot_id  = x['UniprotID']
    ref_smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(i)) for i in ref_map[uniprot_id + '_scaffold'].values() if i != '']
    Finded_Smiles = [i[1:-1] for i in x.Finded_Scaffolds[1:-1].split(', ') if i != '']
        # Vectorized calculation of similarities
    ref_mols = [Chem.MolFromSmiles(smiles) for smiles in ref_smiles if smiles != '']
    valid_mols = [mol for mol in ref_mols if mol is not None]
    ref_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in valid_mols]
## 找到ref_smiles 和训练数据分子的相似性最高的分子
    max_similarities = []
    for ref_Finded_smiles in Finded_Smiles:
        if ref_Finded_smiles == '':
            # max_similarities.append(0.0)
            continue
            
        ref_Finded_mol = Chem.MolFromSmiles(ref_Finded_smiles)
        if ref_Finded_mol is None:
            continue
            
        ref_Finded_fp = AllChem.GetMorganFingerprintAsBitVect(ref_Finded_mol, 2, nBits=2048)
        
        similarities = [DataStructs.TanimotoSimilarity(ref_Finded_fp, fp) for fp in ref_fps]
        
        max_similarities.append(max(similarities))
    
    return max_similarities

In [ ]:
scaffold_molecule_pd1_seen_find_smiles = scaffold_molecule_pd1_seen[scaffold_molecule_pd1_seen['Finded_Smiles_Num'] != 0]
scaffold_molecule_pd1_seen_find_smiles['Max_Similarity'] = scaffold_molecule_pd1_seen_find_smiles.apply(lambda x: get_max_smi_to_trainset(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset), axis=1)
scaffold_molecule_pd1_seen_find_smiles = scaffold_molecule_pd1_seen_find_smiles.groupby('ModelName').agg(list)[['Max_Similarity']].reset_index()#.apply(lambda x: [item for sublist in x['Max_Similarity'] for item in sublist], axis=1)
scaffold_molecule_pd1_seen_find_smiles['Max_Similarity'] = scaffold_molecule_pd1_seen_find_smiles['Max_Similarity'].apply(lambda x: [item for sublist in x for item in sublist])

In [ ]:
# Transform the data for boxplot visualization
plot_data = []
for _, row in scaffold_molecule_pd1_seen_find_smiles.iterrows():
    model_name = row['ModelName']
    similarities =row['Max_Similarity']
    for sim in similarities:
        plot_data.append({'ModelName': model_name, 'Similarity': sim})

plot_df = pd.DataFrame(plot_data)

# Transform the data for boxplot visualization


# Create a boxplot for the similarity data
plt.figure(figsize=(12, 8))
ax = sns.boxplot(data=plot_df, x='ModelName', y='Similarity')
plt.xticks(rotation=45, ha='right')
plt.title('Distribution of Maximum Similarity to Training Set')
plt.ylabel('Tanimoto Similarity')
plt.xlabel('Model Name')

# Add sample size annotations
for i, model in enumerate(plot_df['ModelName'].unique()):
    model_data = plot_df[plot_df['ModelName'] == model]
    n = len(model_data)
    ax.text(i, -0.1, f'n={n}', ha='center', va='top', transform=ax.get_xaxis_transform())

plt.tight_layout()
plt.show()

In [ ]:
# Create a scatter plot for the similarity data
plt.figure(figsize=(12, 8))
color_list = sns.color_palette("deep", n_colors=10)
model_names = sorted(scaffold_molecule_pd_seen['ModelName'].unique().tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}
model_colors = [color_map[name] for name in plot_df['ModelName']]
# Create scatter plot with jitter for better visualization
for i, model in enumerate(plot_df['ModelName'].unique()):
    model_data = plot_df[plot_df['ModelName'] == model]['Similarity']
    # Add small random jitter to x-coordinates for better visibility
    x_jitter = np.random.normal(i, 0.04, size=len(model_data))
    plt.scatter(x_jitter, model_data, alpha=0.6, s=50, color=color_map[model])

plt.xticks(range(len(plot_df['ModelName'].unique())), plot_df['ModelName'].unique(), rotation=45, ha='right')
plt.title('Distribution of Maximum Similarity to Training Set')
plt.ylabel('Max Tanimoto Similarity between Found Smiles and Training Set')
plt.xlabel('Model Name')

plt.tight_layout()

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/mol_scaffold_recovery_seen_protein(Max Tanimoto Similarity between Found Smiles and Training Set).svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()



## compute smilarity between generated active scaffold and scaffold in trainset

In [ ]:
scaffold_molecule_pd1_seen_find_scaffold = scaffold_molecule_pd1_seen[scaffold_molecule_pd1_seen['Finded_Scaffolds_Num'] != 0]
scaffold_molecule_pd1_seen_find_scaffold['Max_Similarity'] = scaffold_molecule_pd1_seen_find_scaffold.apply(lambda x: get_max_scaffold_smi_to_trainset(x, crossdock2020_duplicated_uniprotId_map_smiles_in_trainset), axis=1)
scaffold_molecule_pd1_seen_find_scaffold = scaffold_molecule_pd1_seen_find_scaffold.groupby('ModelName').agg(list)[['Max_Similarity']].reset_index()#.apply(lambda x: [item for sublist in x['Max_Similarity'] for item in sublist], axis=1)
scaffold_molecule_pd1_seen_find_scaffold['Max_Similarity'] = scaffold_molecule_pd1_seen_find_scaffold['Max_Similarity'].apply(lambda x: [item for sublist in x for item in sublist])



In [ ]:
scaffold_molecule_pd1_seen_find_scaffold

In [ ]:
# Transform the data for boxplot visualization
plot_data = []
for _, row in scaffold_molecule_pd1_seen_find_scaffold.iterrows():
    model_name = row['ModelName']
    similarities = row['Max_Similarity']
    for sim in similarities:
        plot_data.append({'ModelName': model_name, 'Similarity': sim})

plot_df_scaffold = pd.DataFrame(plot_data)




# Create a boxplot for the similarity data
plt.figure(figsize=(12, 8))
ax = sns.boxplot(data=plot_df_scaffold, x='ModelName', y='Similarity')
plt.xticks(rotation=45, ha='right')
plt.title('Distribution of Maximum Similarity to Training Set')
plt.ylabel('Tanimoto Similarity')
plt.xlabel('Model Name')

plt.tight_layout()
plt.show()

In [ ]:
# Create a scatter plot for the similarity data
plt.figure(figsize=(12, 8))
color_list = sns.color_palette("deep", n_colors=10)
model_names = sorted(scaffold_molecule_pd_seen['ModelName'].unique().tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}
model_colors = [color_map[name] for name in plot_df['ModelName']]
# Create scatter plot with jitter for better visualization
for i, model in enumerate(plot_df_scaffold['ModelName'].unique()):
    model_data = plot_df_scaffold[plot_df_scaffold['ModelName'] == model]['Similarity']
    # Add small random jitter to x-coordinates for better visibility
    x_jitter = np.random.normal(i, 0.04, size=len(model_data))
    plt.scatter(x_jitter, model_data, alpha=0.6, s=50,color=color_map[model])

plt.xticks(range(len(plot_df_scaffold['ModelName'].unique())), plot_df_scaffold['ModelName'].unique(), rotation=45, ha='right')
plt.title('Distribution of Maximum Similarity to Training Set')
plt.ylabel('Max Tanimoto Similarity between Found Scaffolds and Training Set')
plt.xlabel('Model Name')

plt.tight_layout()

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/mol_scaffold_recovery_seen_protein(Max Tanimoto Similarity between Found Scaffolds and Training Set).svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()

## 计算和见过的阳性骨架一样的比例

In [ ]:

same_scaffold= plot_df_scaffold[plot_df_scaffold['Similarity']==1].groupby('ModelName').agg(list)['Similarity'].apply(lambda x: len(x)).reset_index()
same_scaffold['all'] = plot_df_scaffold.groupby('ModelName').agg(list)['Similarity'].apply(lambda x: len(x)).reset_index()['Similarity']

In [ ]:
same_scaffold.columns = ['ModelName','Same_Scaffold','All_Scaffold']
same_scaffold['same_ratio'] =  same_scaffold['Same_Scaffold']/same_scaffold['All_Scaffold']
same_scaffold = same_scaffold.sort_values(by='same_ratio', ascending=False)
same_scaffold

## 统计0.6-0.8区间的比例

In [ ]:
scaffold_6_8= plot_df_scaffold[(plot_df_scaffold['Similarity']>0.6)&(plot_df_scaffold['Similarity']<0.8)].groupby('ModelName').agg(list)['Similarity'].apply(lambda x: len(x)).reset_index()
scaffold_6_8['all'] = plot_df_scaffold.groupby('ModelName').agg(list)['Similarity'].apply(lambda x: len(x)).reset_index()['Similarity']
scaffold_6_8.columns = ['ModelName','scaffold_6_8','All_Scaffold']
scaffold_6_8['same_ratio'] =  scaffold_6_8['scaffold_6_8']/scaffold_6_8['All_Scaffold']
scaffold_6_8 = scaffold_6_8.sort_values(by='same_ratio', ascending=False)
scaffold_6_8

In [ ]:
# Define similarity intervals
intervals = [(0.0, 0.2), (0.2, 0.4), (0.4, 0.6), (0.6, 0.8), (0.8, 1.0), (1.0, 1.0)]
interval_labels = ['0.0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0', '1.0']
color_list = sns.color_palette("deep", n_colors=10)
model_names = sorted(scaffold_molecule_pd_seen['ModelName'].unique().tolist())
color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}
model_colors = [color_map[name] for name in plot_df['ModelName']]
# Calculate proportions for each model and interval
proportion_data = []
for model in plot_df_scaffold['ModelName'].unique():
    model_data = plot_df_scaffold[plot_df_scaffold['ModelName'] == model]['Similarity']
    total_count = len(model_data)
    
    for i, (low, high) in enumerate(intervals):
        if i == len(intervals) - 1:  # For exact 1.0 matches
            count = sum(model_data == 1.0)
        else:
            count = sum((model_data >= low) & (model_data < high))
        
        proportion = count / total_count if total_count > 0 else 0
        proportion_data.append({
            'ModelName': model,
            'Interval': interval_labels[i],
            'Count': count,
            'Proportion': proportion,
            'Total': total_count
        })

proportion_df = pd.DataFrame(proportion_data)

# Create a pivot table for easier plotting
pivot_df = proportion_df.pivot(index='ModelName', columns='Interval', values='Proportion').fillna(0)

# Plot stacked bar chart
plt.figure(figsize=(14, 8))
colors = sns.color_palette("Set2", n_colors=len(interval_labels))

# Create stacked bar chart
bottom = np.zeros(len(pivot_df))
bars = []
for i, interval in enumerate(interval_labels):
    if interval in pivot_df.columns:
        bar = plt.bar(pivot_df.index, pivot_df[interval], bottom=bottom, 
                     label=interval, color=colors[i], alpha=0.5)
        bars.append(bar)
        bottom += pivot_df[interval]

plt.xlabel('Model Name')
plt.ylabel('Proportion')
plt.title('Distribution of Scaffold Similarity to Training Set by Intervals', pad=20)
plt.legend(title='Similarity Intervals', bbox_to_anchor=(1, 1), loc='upper left')
plt.xticks(rotation=45, ha='right')

# Add sample size annotations
for i, model in enumerate(pivot_df.index):
    total = proportion_df[proportion_df['ModelName'] == model]['Total'].iloc[0]
    plt.text(i, 1.05, f'n={total}', ha='center', va='bottom')

# plt.tight_layout()
# plt.show()

# plt.text(i, -0.1, f'n={n}', ha='center', va='top', transform=plt.gca().get_xaxis_transform())
plt.tight_layout()

plt.savefig('/home/data-house-01/cdhofficial/MolGens/TestSample_Denovo/Round1/De_novo_Results/mol_scaffold_recovery_seen_protein(Distribution of Max Scaffold Similarity to Training Set by Intervals).svg',bbox_inches='tight',dpi=660,format='svg')
plt.show()